In [7]:
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, IntegerType, TimestampType
from pyspark.sql.functions import sum,avg,max,count, hour, when, col, date_format, month, to_date

# from mrjob.job import MRJob

In [8]:
# vars
_DATA_OG_PATH = 'data/original/'
_DATA_AGG_PATH = 'data/aggregated/'
_YEARS = ['2019', '2020', '2021', '2022']
_MONTHS = [str('0' + str(m))[-2:] for m in range(1, 13)]

### Move all files to folders sorted by location

In [9]:
# move files to 'year' folders
# import os

# years = ['2019', '2020', '2021', '2022']
# path = 'data/data_unzipped/'
# all_files = os.listdir(path)
# files_year = {year:[f for f in all_files if os.path.isfile(os.path.join(path, f)) and year in f] for year in years}

# for year in files_year:
#     tmp = ' '.join(map(lambda x: os.path.join(path, x), files_year[year]))
#     os.system(f'mv {tmp} {os.path.join(path, year)}')

### Transform all files - extract 'month', 'year', 'total_distance'

In [10]:
# def transform_and_save(year = '2019'):
#     assert year in _YEARS
#     files = listdir(join(_DATA_PATH, year))
#     for path in files:
        

In [11]:
from calendar import month
from xml.dom.expatbuilder import FilterVisibilityController


def group_and_save(filename: str):

    stats = {}

    spark = SparkSession.builder.master("local[*]").appName('citibike').getOrCreate()
    try:
        data = spark.read.option("header",True).csv(join(_DATA_OG_PATH, '2019', filename))
        print('---')                        
        print(f'File name: {FilterVisibilityController]}')
        print('---')   
    except Exception as e:
        print('File read error')
        print(e)
        return None

    stats['count'] = data.count()

    data = data.withColumn('month', month(to_date(col('starttime'))))

    data = data.withColumn('tripduration', data['tripduration'].cast(IntegerType())) \
                            .withColumn('birth year', data['birth year'].cast(IntegerType())) \
                            .withColumn('gender', data['gender'].cast(IntegerType())) \
                            .withColumn('starttime', data['starttime'].cast(TimestampType())) \
                            .withColumn('stoptime', data['stoptime'].cast(TimestampType())) 
                            
    data = data.withColumn('startTimeHour',hour(col('starttime')))

                         
    print('Schema')
    print(data.printSchema())
    print('---')                        


    # time of day extract
    data = data.withColumn('startTimePeriod',
                     when(hour(col('starttime')) < 6, 'midnight')\
                    .when((hour(col('starttime')) >= 6) & (hour(col('starttime')) < 12), 'morning')\
                    .when((hour(col('starttime')) >= 12) & (hour(col('starttime')) < 17), 'afternoon')\
                    .when(hour(col('starttime')) >= 17, 'evening')
                    )
    # aggregate
    try:
        groupTrip = data.groupBy(['start station name', 'end station name', 'usertype', 'startTimeHour', 'month']) \
                        .agg(count("*").alias("count"), \
                             sum("tripduration").alias("totaltripduration"))

        groupTrip.repartition(1)\
        .write.format("csv")\
        .option("header", "true")\
        .save(join(_DATA_AGG_PATH, 'TRIPSUMM_' + filename))
        print('AGGTRIP File saved')
        groupTrip.drop('*')
    except Exception as e:
        print('AGG TRIP: File save error')
        print(e)

    try:
        groupDemo = data.groupBy(['birth year', 'gender', 'usertype', 'startTimePeriod', 'month']) \
                        .agg(count("*").alias("count"), \
                             sum("tripduration").alias("totaltripduration"))

        groupDemo.repartition(1)\
        .write.format("csv")\
        .option("header", "true")\
        .save(join(_DATA_AGG_PATH, 'TRIPSUMM_' + filename))
        print('AGGDEMO File saved')
        groupDemo.drop('*')
    except Exception as e:
        print('File save error')
        print(e)

    spark._sc.stop()
    print('='*10)

    return stats
    



In [12]:
files2019 = listdir(join(_DATA_OG_PATH, '2019'))
counts2019 = {}
for filename in files2019:
    stats = group_and_save(filename)
    if stats:
        counts2019[filename] = stats['count']
    else:
        pass

---
File name: {files2019[0]}
---


TypeError: formatmonth() missing 1 required positional argument: 'themonth'

In [ ]:
err

In [ ]:
spark = SparkSession.builder.master("local[*]").appName('citibike').getOrCreate()
try:
    data = spark.read.option("header",True).csv(join(_DATA_OG_PATH, '2019', files2019[0]))
except Exception as e:
    print('File read error')
    print(e)

data = data.withColumn('month', month(to_date(col('starttime'))))
data.select('month').show(1)

+-----+
|month|
+-----+
|    8|
+-----+
only showing top 1 row

